In [2]:
import mne
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import signal
from scipy.stats import entropy
import pywt
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

# =================== HÀM TÍNH FEATURE ===================

def compute_band_power(data, sfreq):
    freqs, psd = signal.welch(data, sfreq, nperseg=min(256, data.shape[-1]))
    bands = {'delta': (0.5,4), 'theta': (4,8), 'alpha': (8,13),
             'beta': (13,30), 'gamma': (30,45)}
    power_features = {}
    for band, (low, high) in bands.items():
        idx = np.logical_and(freqs >= low, freqs <= high)
        power_features[band] = np.trapz(psd[:, idx], freqs[idx], axis=1)
    return power_features

def compute_spectral_entropy(data, sfreq):
    freqs, psd = signal.welch(data, sfreq, nperseg=min(256, data.shape[-1]))
    psd_norm = psd / np.sum(psd, axis=1, keepdims=True)
    return -np.sum(psd_norm * np.log2(psd_norm + 1e-10), axis=1)

def compute_wavelet_features(data, wavelet='db4', level=3):
    n_channels = data.shape[0]
    features = {'coeffs_mean': [], 'coeffs_energy': []}
    for ch in range(n_channels):
        coeffs = pywt.wavedec(data[ch], wavelet, level=level)
        features['coeffs_mean'].append([np.mean(np.abs(c)) for c in coeffs])
        features['coeffs_energy'].append([np.sum(c**2) for c in coeffs])
    for k in features:
        features[k] = np.array(features[k])
    return features

def combine_features(bp, spec_ent, wave_feats, ch_names):
    all_feats = []
    for i, ch in enumerate(ch_names):
        ch_feats = []
        for band in bp:
            ch_feats.append(bp[band][i])
        ch_feats.append(spec_ent[i])
        for lvl in range(wave_feats['coeffs_mean'].shape[1]):
            ch_feats.append(wave_feats['coeffs_mean'][i][lvl])
            ch_feats.append(wave_feats['coeffs_energy'][i][lvl])
        all_feats.extend(ch_feats)
    return all_feats

# =================== XỬ LÝ 1 SUBJECT ===================

def process_subject(preproc_file):
    subject = preproc_file.parent.parent.name 
    
    # --- Tìm file events của subject ---
    events_file = Path(f"data/EEG/eeg/tsv/{subject}_task-oddball_events.tsv")
    
    if not events_file.exists():
        print(f"Không tìm thấy events cho {subject}")
        return []

    # ---- Load events ----
    events_tsv = pd.read_csv(events_file, sep="\t")
    try:
        raw = mne.io.read_raw_fif(preproc_file, preload=True, verbose=False)
    except Exception as e:
        print(f"Lỗi khi đọc {preproc_file}: {e}")
        return []
    sfreq = raw.info['sfreq']
    ch_names = raw.ch_names

    results = []

    for idx, row in events_tsv.iterrows():
        onset_sample = int(row["onset"])
        label = row["event_type"] 

        segment_samples = int(1.0 * sfreq)
        data = raw.get_data(start=onset_sample, stop=onset_sample + segment_samples)


        bp = compute_band_power(data, sfreq)
        spec_ent = compute_spectral_entropy(data, sfreq)
        wave_feats = compute_wavelet_features(data)
        feats = combine_features(bp, spec_ent, wave_feats, ch_names)

        results.append([subject, preproc_file.name, idx, label] + feats)

    return results

# =================== BATCH PROCESS ===================

def extract_features_batch(derivatives_path, output_file='eeg_features.csv', n_jobs=-1):
    derivatives_path = Path(derivatives_path)
    files = list(derivatives_path.rglob('*_desc-preproc_eeg.fif'))
    print(f"Tìm thấy {len(files)} file EEG")

    # Song song hóa
    all_results = Parallel(n_jobs=n_jobs)(
        delayed(process_subject)(f) for f in files
    )

    flat = [item for sublist in all_results for item in sublist]

    df = pd.DataFrame(flat)
    df = df.rename(columns={0: "subject", 1: "file", 2: "trial", 3: "label"})
    feature_cols = [f"f{i}" for i in range(df.shape[1] - 4)]
    df.columns = ["subject", "file", "trial", "label"] + feature_cols

    df.to_csv(output_file, index=False)
    print("DONE →", output_file, "shape:", df.shape)

    return df

# =================== RUN ===================

if __name__ == "__main__":
    extract_features_batch(
        "data/EEG/derivatives/preprocessing",
        output_file="optimized_eeg_features.csv",
        n_jobs=-1
    )

Tìm thấy 42 file EEG
DONE → optimized_eeg_features.csv shape: (30396, 1782)


In [12]:
import pandas as pd
tsv = pd.read_csv("eeg_features.csv")


In [13]:
tsv.head()

,subject,file,trial,label,f0,f1,f2,f3,f4,f5,...,f1768,f1769,f1770,f1771,f1772,f1773,f1774,f1775,f1776,f1777
0,sub-01,sub-01_task-oddball_desc-preproc_eeg.fif,0,S 5,0.0,0.0,0.0,2.810764e-11,5.100409e-12,1.207314,...,2.839575e-12,1.260830,0.000101,2.561104e-06,9.135965e-07,1.814907e-10,6.672152e-08,4.236073e-12,2.901080e-09,1.476699e-14
1,sub-01,sub-01_task-oddball_desc-preproc_eeg.fif,1,S 5,0.0,0.0,0.0,1.188307e-11,5.867867e-12,1.148442,...,4.879814e-12,1.186929,0.000072,1.774518e-06,1.054273e-06,2.410315e-10,7.116892e-08,2.195702e-12,3.421914e-09,1.024458e-14
2,sub-01,sub-01_task-oddball_desc-preproc_eeg.fif,2,S 5,0.0,0.0,0.0,1.324512e-11,8.520660e-12,3.392780,...,3.554239e-12,3.292150,0.000012,2.908802e-08,1.034735e-06,2.683197e-10,8.124062e-08,8.471733e-12,3.411116e-09,2.550000e-14
3,sub-01,sub-01_task-oddball_desc-preproc_eeg.fif,3,S 5,0.0,0.0,0.0,1.258177e-11,8.381647e-12,1.492794,...,3.724248e-12,1.537859,0.000059,1.368634e-06,9.047409e-07,1.643513e-10,7.403810e-08,8.404831e-12,3.283984e-09,3.630522e-14
4,sub-01,sub-01_task-oddball_desc-preproc_eeg.fif,4,S 7,0.0,0.0,0.0,3.355817e-11,7.498204e-12,1.444070,...,2.766799e-12,1.493626,0.000133,4.057098e-06,7.838209e-07,1.218728e-10,5.154912e-08,1.252141e-12,2.464448e-09,6.117372e-15


In [6]:
import pandas as pd
tsv = pd.read_csv("data/EEG/sub-01/eeg/sub-01_task-oddball_events.tsv", sep="\t")
tsv["onset"].max()


1308857

In [7]:
raw = mne.io.read_raw_fif("data/EEG/derivatives/preprocessing/sub-01/eeg/sub-01_task-oddball_desc-preproc_eeg.fif", preload=True)
raw.n_times


Opening raw data file data/EEG/derivatives/preprocessing/sub-01/eeg/sub-01_task-oddball_desc-preproc_eeg.fif...
    Range : 0 ... 1371319 =      0.000 ...  1371.319 secs
Ready.
Reading 0 ... 1371319  =      0.000 ...  1371.319 secs...


1371320

V2

In [ ]:
"""
Đoạn mã này thực hiện trích xuất đặc trưng EEG tập trung vào các kênh ROI liên quan đến phản ứng Oddball/P300 (như Fz, Cz, Pz,…).
Với mỗi sự kiện, tín hiệu EEG được cắt trong cửa sổ từ –200 ms đến 800 ms, 
chuẩn hóa baseline rồi tính toán các nhóm đặc trưng chính gồm năng lượng dải tần (band power), 
entropy phổ và đặc trưng wavelet đa mức. Các đặc trưng theo từng kênh được ghép lại thành một vector duy nhất, 
tạo thành bộ feature tối ưu phục vụ cho phân tích và mô hình học máy trong các nghiên cứu Oddball/P300.
"""


import mne
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import signal
import pywt
from joblib import Parallel, delayed
import warnings

warnings.filterwarnings('ignore')

# =================== CẤU HÌNH KÊNH QUAN TRỌNG (ODDBALL ROI) ===================
# Danh sách các kênh tiêu chuẩn phản ánh tốt nhất P300/Oddball
# Bạn cần kiểm tra xem tên kênh trong file raw của bạn có khớp format này không (ví dụ 'Cz' hay 'EEG001')
ROI_CHANNELS = ['Fz', 'Cz', 'Pz', 'Oz', 'P3', 'P4', 'C3', 'C4'] 

# =================== HÀM TÍNH FEATURE (GIỮ NGUYÊN) ===================
def compute_band_power(data, sfreq):
    # Giảm nperseg nếu đoạn dữ liệu ngắn hơn 1s
    n_samples = data.shape[-1]
    freqs, psd = signal.welch(data, sfreq, nperseg=min(128, n_samples)) 
    bands = {'delta': (0.5,4), 'theta': (4,8), 'alpha': (8,13),
             'beta': (13,30), 'gamma': (30,45)}
    power_features = {}
    for band, (low, high) in bands.items():
        idx = np.logical_and(freqs >= low, freqs <= high)
        if np.sum(idx) == 0: # Tránh lỗi nếu không có tần số nào lọt vào band
            power_features[band] = np.zeros(data.shape[0])
        else:
            power_features[band] = np.trapz(psd[:, idx], freqs[idx], axis=1)
    return power_features

def compute_spectral_entropy(data, sfreq):
    n_samples = data.shape[-1]
    freqs, psd = signal.welch(data, sfreq, nperseg=min(128, n_samples))
    psd_norm = psd / np.sum(psd, axis=1, keepdims=True)
    return -np.sum(psd_norm * np.log2(psd_norm + 1e-10), axis=1)

def compute_wavelet_features(data, wavelet='db4', level=3):
    n_channels = data.shape[0]
    features = {'coeffs_mean': [], 'coeffs_energy': []}
    for ch in range(n_channels):
        # Mode 'periodic' hoặc 'symmetric' thường tốt cho tín hiệu ngắn
        coeffs = pywt.wavedec(data[ch], wavelet, level=level, mode='symmetric')
        features['coeffs_mean'].append([np.mean(np.abs(c)) for c in coeffs])
        features['coeffs_energy'].append([np.sum(c**2) for c in coeffs])
    for k in features:
        features[k] = np.array(features[k])
    return features

def combine_features(bp, spec_ent, wave_feats, ch_names):
    all_feats = []
    for i, ch in enumerate(ch_names):
        ch_feats = []
        for band in bp:
            ch_feats.append(bp[band][i])
        ch_feats.append(spec_ent[i])
        for lvl in range(wave_feats['coeffs_mean'].shape[1]):
            ch_feats.append(wave_feats['coeffs_mean'][i][lvl])
            ch_feats.append(wave_feats['coeffs_energy'][i][lvl])
        all_feats.extend(ch_feats)
    return all_feats

# =================== XỬ LÝ 1 SUBJECT ===================

def process_subject(preproc_file):
    subject = preproc_file.parent.parent.name 
    
    # --- Tìm file events ---
    events_file = Path(f"data/EEG/eeg/tsv/{subject}_task-oddball_events.tsv")
    if not events_file.exists():
        return []

    events_tsv = pd.read_csv(events_file, sep="\t")
    try:
        raw = mne.io.read_raw_fif(preproc_file, preload=True, verbose=False)
        raw.set_annotations(None)
    except Exception as e:
        print(f"Lỗi khi đọc file {preproc_file}: {e}")
        return []
        
    sfreq = raw.info['sfreq']
    
    # --- [NEW] CHỌN KÊNH (CHANNEL SELECTION) ---
    # Lọc lấy các kênh có trong danh sách ROI và tồn tại trong dữ liệu
    available_channels = [ch for ch in ROI_CHANNELS if ch in raw.ch_names]
    if len(available_channels) == 0:
        # Fallback: Nếu tên kênh không khớp chuẩn 10-20, lấy tạm 10 kênh đầu hoặc báo lỗi
        print(f"Warning: Không tìm thấy kênh ROI chuẩn cho {subject}. Dùng tất cả kênh.")
        available_channels = raw.ch_names
    
    # Chỉ lấy dữ liệu của các kênh đã chọn để xử lý
    # Lưu ý: Ta không drop kênh trong 'raw' để tránh lỗi bộ nhớ, ta sẽ dùng tham số 'picks' khi get_data
    
    results = []

    # --- [NEW] THIẾT LẬP CỬA SỔ THỜI GIAN ---
    tmin, tmax = -0.2, 0.8
    
    for idx, row in events_tsv.iterrows():
        # Lưu ý: 'onset' trong BIDS tsv thường là GIÂY. Cần nhân với sfreq nếu nó là float.
        # Nếu file tsv của bạn onset là index mẫu (int), hãy giữ nguyên logic cũ.
        # Dưới đây giả định row["onset"] là index mẫu (như code gốc của bạn):
        center_sample = int(row["onset"]) 
        
        # Tính index bắt đầu và kết thúc
        start_sample = center_sample + int(tmin * sfreq)
        stop_sample = center_sample + int(tmax * sfreq)

        # Kiểm tra biên (tránh lấy chỉ số âm hoặc vượt quá độ dài)
        if start_sample < 0 or stop_sample > raw.n_times:
            continue

        # --- [NEW] CẮT DỮ LIỆU VỚI PICKS ---
        # picks=available_channels: Chỉ lấy dữ liệu các kênh quan trọng
        data = raw.get_data(picks=available_channels, start=start_sample, stop=stop_sample)
        
        # Optional: BASELINE CORRECTION (Trừ đi trung bình của đoạn -0.2 đến 0)
        # Đoạn baseline là từ đầu (index 0) đến mốc 0s (tương ứng 0.2s đầu tiên)
        baseline_samples = int(0.2 * sfreq)
        baseline_mean = np.mean(data[:, :baseline_samples], axis=1, keepdims=True)
        data = data - baseline_mean 

        # Tính features
        bp = compute_band_power(data, sfreq)
        spec_ent = compute_spectral_entropy(data, sfreq)
        wave_feats = compute_wavelet_features(data)
        
        # Truyền tên các kênh đã lọc vào để combine
        feats = combine_features(bp, spec_ent, wave_feats, available_channels)

        results.append([subject, preproc_file.name, idx, row["event_type"]] + feats)

    return results

# =================== BATCH PROCESS ===================

def extract_features_batch(derivatives_path, output_file='eeg_features_roi.csv', n_jobs=-1):
    derivatives_path = Path(derivatives_path)
    files = list(derivatives_path.rglob('*_desc-preproc_eeg.fif'))
    print(f"Tìm thấy {len(files)} file EEG")

    all_results = Parallel(n_jobs=n_jobs)(
        delayed(process_subject)(f) for f in files
    )

    flat = [item for sublist in all_results for item in sublist]

    if not flat:
        print("Không trích xuất được dữ liệu nào!")
        return pd.DataFrame()

    df = pd.DataFrame(flat)
    
    # Tạo tên cột feature động dựa trên số lượng feature trích xuất được
    n_meta_cols = 4 # subject, file, trial, label
    n_feature_cols = df.shape[1] - n_meta_cols
    feature_cols = [f"feat_{i}" for i in range(n_feature_cols)]
    
    df.columns = ["subject", "file", "trial", "label"] + feature_cols
    df.to_csv(output_file, index=False)
    print("DONE →", output_file, "shape:", df.shape)
    return df

if __name__ == "__main__":
    extract_features_batch(
        "data/EEG/derivatives/preprocessing",
        output_file="optimized_oddball_features.csv",
        n_jobs=-1
    )

Tìm thấy 42 file EEG
DONE → optimized_oddball_features.csv shape: (30396, 116)


V3

In [1]:
"""
Code trích xuất 3 nhóm feature hiệu quả nhất cho Oddball paradigm:
1. ERP peak/area (P300 components)
2. CSP (Common Spatial Patterns) - thay thế xDAWN, nhẹ hơn
3. Covariance → Tangent space features

Memory-optimized version
"""

import mne
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.integrate import simpson
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

# =================== CSP IMPLEMENTATION ===================

def fit_csp(X, y, n_components=4):
    """
    Common Spatial Pattern - memory efficient alternative to xDAWN
    Args:
        X: (n_trials, n_channels, n_times)
        y: (n_trials,) binary labels (0 or 1)
    Returns:
        filters: (n_components, n_channels)
    """
    n_channels = X.shape[1]
    
    # Separate by class
    X_class0 = X[y == 0]
    X_class1 = X[y == 1]
    
    if len(X_class0) == 0 or len(X_class1) == 0:
        # Fallback: return identity-like filters
        filters = np.eye(n_channels)[:n_components]
        return filters
    
    # Compute average covariance for each class (memory efficient)
    def avg_covariance(X_subset):
        cov_sum = np.zeros((n_channels, n_channels))
        n_samples = min(30, len(X_subset))  # Limit to 30 trials
        for i in range(n_samples):
            cov_sum += np.cov(X_subset[i])
        return cov_sum / n_samples
    
    C0 = avg_covariance(X_class0)
    C1 = avg_covariance(X_class1)
    
    # Composite covariance
    C_composite = C0 + C1
    
    # Regularization
    C_composite += 1e-6 * np.eye(n_channels)
    
    # Eigenvalue decomposition
    try:
        eigenvalues, eigenvectors = np.linalg.eigh(C0, C_composite)
        # Sort by eigenvalues
        idx = np.argsort(eigenvalues)[::-1]
        # Take top and bottom components (most discriminative)
        selected_idx = np.concatenate([idx[:n_components//2], idx[-n_components//2:]])
        filters = eigenvectors[:, selected_idx].T
    except:
        # Fallback
        U, s, Vt = np.linalg.svd(C0, full_matrices=False)
        filters = U[:, :n_components].T
    
    return filters

def compute_csp_features(X, y, n_components=4):
    """
    Apply CSP filters and extract features
    Args:
        X: (n_trials, n_channels, n_times)
        y: (n_trials,) binary labels
    """
    filters = fit_csp(X, y, n_components)
    
    # Transform data
    X_csp = np.zeros((X.shape[0], n_components, X.shape[2]))
    for trial in range(X.shape[0]):
        X_csp[trial] = filters @ X[trial]
    
    # Extract features per component
    features = {}
    for comp in range(n_components):
        comp_data = X_csp[:, comp, :]
        features[f'csp_c{comp}_var'] = np.var(comp_data, axis=1)
        features[f'csp_c{comp}_mean'] = np.mean(comp_data, axis=1)
        features[f'csp_c{comp}_max'] = np.max(comp_data, axis=1)
    
    return features

# =================== ERP FEATURES ===================

def compute_erp_features(data, sfreq, p300_window=(0.25, 0.5)):
    """
    Tính P300 peak amplitude và area
    """
    start_idx = int(p300_window[0] * sfreq)
    end_idx = int(p300_window[1] * sfreq)
    
    # Handle edge cases
    if end_idx > data.shape[1]:
        end_idx = data.shape[1]
    if start_idx >= end_idx:
        start_idx = 0
    
    p300_segment = data[:, start_idx:end_idx]
    
    features = {}
    features['p300_peak'] = np.max(p300_segment, axis=1)
    features['p300_area'] = simpson(p300_segment, axis=1)
    features['p300_mean'] = np.mean(p300_segment, axis=1)
    
    return features

def extract_trial_features(data, sfreq, ch_names):
    """Extract ERP features for 1 trial"""
    features = []
    
    erp_feats = compute_erp_features(data, sfreq)
    for feat_name, feat_vals in erp_feats.items():
        features.extend(feat_vals)
    
    return features

# =================== TANGENT SPACE FEATURES ===================

def compute_tangent_features(epochs_data, max_trials=100):
    """
    Riemannian tangent space features - memory limited
    """
    # Limit number of trials to prevent memory issues
    if epochs_data.shape[0] > max_trials:
        indices = np.random.choice(epochs_data.shape[0], max_trials, replace=False)
        indices.sort()
        epochs_subset = epochs_data[indices]
    else:
        epochs_subset = epochs_data
        indices = np.arange(epochs_data.shape[0])
    
    try:
        # Estimate covariance matrices
        cov = Covariances(estimator='oas')  # OAS is faster than lwf
        cov_matrices = cov.fit_transform(epochs_subset)
        
        # Project to tangent space
        ts = TangentSpace(metric='riemann')
        tangent_feats = ts.fit_transform(cov_matrices)
        
        # Expand back to full size if we sampled
        if len(indices) < epochs_data.shape[0]:
            full_tangent = np.zeros((epochs_data.shape[0], tangent_feats.shape[1]))
            full_tangent[indices] = tangent_feats
            # Fill missing with mean
            mean_feat = np.mean(tangent_feats, axis=0)
            mask = np.ones(epochs_data.shape[0], dtype=bool)
            mask[indices] = False
            full_tangent[mask] = mean_feat
            return full_tangent
        
        return tangent_feats
    except:
        # Fallback: return zeros
        return np.zeros((epochs_data.shape[0], 10))

# =================== GROUP FEATURES ===================

def extract_group_features(epochs_data, labels):
    """
    Extract CSP and tangent features
    """
    n_trials = epochs_data.shape[0]
    
    # Convert labels to binary
    unique_labels = np.unique(labels)
    y = np.array([0 if label == unique_labels[0] else 1 for label in labels])
    
    # CSP features
    csp_feats = compute_csp_features(epochs_data, y, n_components=4)
    
    # Tangent space features (with memory limit)
    tangent_feats = compute_tangent_features(epochs_data, max_trials=100)
    
    # Organize by trial
    trial_features = {}
    for trial_idx in range(n_trials):
        feats = []
        
        # CSP features
        for key in sorted(csp_feats.keys()):
            feats.append(csp_feats[key][trial_idx])
        
        # Tangent features
        feats.extend(tangent_feats[trial_idx])
        
        trial_features[trial_idx] = feats
    
    return trial_features

# =================== PROCESS SUBJECT ===================

def process_subject(preproc_file):
    """Process one subject"""
    subject = preproc_file.parent.parent.name
    
    # Find events file
    events_file = Path(f"data/EEG/eeg/tsv/{subject}_task-oddball_events.tsv")
    
    if not events_file.exists():
        print(f"⚠ Không tìm thấy events cho {subject}")
        return []
    
    try:
        # Load data
        events_tsv = pd.read_csv(events_file, sep="\t")
        raw = mne.io.read_raw_fif(preproc_file, preload=True, verbose=False)
        sfreq = raw.info['sfreq']
        ch_names = raw.ch_names
        
        # Create events array
        events_array = []
        event_mapping = {}
        next_id = 1
        
        for idx, row in events_tsv.iterrows():
            onset_sample = int(row["onset"])
            event_type = row["event_type"]
            
            if event_type not in event_mapping:
                event_mapping[event_type] = next_id
                next_id += 1
            
            events_array.append([onset_sample, 0, event_mapping[event_type]])
        
        events_array = np.array(events_array)
        
        if len(events_array) == 0:
            return []
        
        event_id_dict = {k: v for k, v in event_mapping.items()}
        
        # Create epochs
        epochs = mne.Epochs(raw, events_array, event_id=event_id_dict,
                            tmin=-0.2, tmax=0.8, baseline=(-0.2, 0),
                            preload=True, verbose=False, 
                            reject_by_annotation=False)
        
        epochs_data = epochs.get_data()
        labels = [events_tsv.iloc[i]["event_type"] for i in range(len(events_tsv))]
        
        # Check length match
        if len(epochs_data) != len(labels):
            min_len = min(len(epochs_data), len(labels))
            epochs_data = epochs_data[:min_len]
            labels = labels[:min_len]
        
        # Extract group features
        group_features = extract_group_features(epochs_data, labels)
        
        results = []
        
        # Process each trial
        for idx, row in events_tsv.iterrows():
            if idx >= len(group_features):
                continue
            
            onset_sample = int(row["onset"])
            label = row["event_type"]
            
            segment_samples = int(1.0 * sfreq)
            
            if onset_sample + segment_samples > raw.n_times:
                continue
            
            data = raw.get_data(start=onset_sample, stop=onset_sample + segment_samples)
            
            # ERP features
            erp_feats = extract_trial_features(data, sfreq, ch_names)
            
            # Combine all features
            all_feats = erp_feats + group_features[idx]
            
            results.append([subject, preproc_file.name, idx, label] + all_feats)
        
        print(f"✓ {subject}: {len(results)} trials")
        return results
        
    except Exception as e:
        print(f"✗ Error processing {subject}: {e}")
        return []

# =================== BATCH PROCESS ===================

def extract_features_batch(derivatives_path, output_file='oddball_top3_features.csv', n_jobs=2):
    """Process all subjects"""
    derivatives_path = Path(derivatives_path)
    files = list(derivatives_path.rglob('*_desc-preproc_eeg.fif'))
    print(f"Tìm thấy {len(files)} file EEG\n")
    
    # Process in parallel
    all_results = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(process_subject)(f) for f in files
    )
    
    # Flatten
    flat = [item for sublist in all_results for item in sublist]
    
    if len(flat) == 0:
        print("⚠ Không có data nào được xử lý!")
        return None
    
    # Create DataFrame
    df = pd.DataFrame(flat)
    df = df.rename(columns={0: "subject", 1: "file", 2: "trial", 3: "label"})
    feature_cols = [f"f{i}" for i in range(df.shape[1] - 4)]
    df.columns = ["subject", "file", "trial", "label"] + feature_cols
    
    df.to_csv(output_file, index=False)
    
    print(f"\n{'='*60}")
    print(f"✓ DONE → {output_file}")
    print(f"  Shape: {df.shape}")
    print(f"  Features:")
    print(f"    - ERP: P300 peak/area/mean × {len(ch_names)} channels")
    print(f"    - CSP: 4 components × 3 stats = 12 features")
    print(f"    - Tangent space: Riemannian covariance features")
    print(f"{'='*60}")
    
    return df

# =================== RUN ===================

if __name__ == "__main__":
    df = extract_features_batch(
        "data/EEG/derivatives/preprocessing",
        output_file="oddball_top3_features.csv",
        n_jobs=2  # Giảm xuống 2 để tránh memory issues
    )

Tìm thấy 42 file EEG



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   36.6s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  1.2min
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:  2.8min
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:  4.0min
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:  6.2min
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  8.2min
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 10.4min
[Parallel(n_jobs=2)]: Done  42 out of  42 | elapsed: 12.1min finished



✓ DONE → oddball_top3_features.csv
  Shape: (29639, 8525)
  Features:


NameError: name 'ch_names' is not defined

In [18]:
import numpy
import scipy
import mne

print(f"Numpy version: {numpy.__version__}")
print(f"MNE version: {mne.__version__}")

Numpy version: 1.26.4
MNE version: 1.10.2
